In [65]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
from tqdm.notebook import tqdm_notebook
from datasets import Dataset
pd.set_option('display.max_rows', None)



tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

def normalize(num):
    return num+1

loading file vocab.txt from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_l

In [66]:
df = pd.read_csv("finetuning.csv").drop('Unnamed: 0',axis =1)
df= df.drop(1005,axis=0)

In [67]:
df.sentiment.value_counts()

 1    649
-1    510
 0    402
Name: sentiment, dtype: int64

In [68]:
df_train = df


In [69]:
df_train['sentiment'] = df_train['sentiment'].apply(normalize)

In [70]:
text_list = list(df_train['tweet'])
labels =list(df_train['sentiment'])

In [71]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TextDataset, DataCollatorWithPadding, Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Encode the text and labels into a Dataset
def encode_examples(text, label):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=False)
    return {'input_ids': inputs['input_ids'][0], 'attention_mask': inputs['attention_mask'][0], 'label': torch.tensor(label)}

encoded_dataset = [encode_examples(text, label) for text, label in zip(text_list, labels)]
dataset = Dataset.from_dict({k: [d[k] for d in encoded_dataset] for k in encoded_dataset[0]})

# Create a data collator to handle padding during training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    gradient_accumulation_steps=1,
    fp16=False,
    seed=42,
)
# Define the accuracy metric for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_tensor = torch.tensor(logits)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    predictions = torch.argmax(logits_tensor, dim=-1)
    accuracy = (predictions == labels_tensor).sum().item() / len(labels)
    return {'accuracy': accuracy}


# Create a Trainer to handle fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('./fine-tuned')


loading file vocab.txt from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_l

Epoch,Training Loss,Validation Loss,Accuracy
1,0.549800,0.254217,0.923767
2,0.147100,0.162070,0.956438
3,0.117100,0.121116,0.969250


***** Running Evaluation *****
  Num examples = 1561
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-196
Configuration saved in ./results/checkpoint-196/config.json
Model weights saved in ./results/checkpoint-196/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-196/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-196/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1561
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-392
Configuration saved in ./results/checkpoint-392/config.json
Model weights saved in ./results/checkpoint-392/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-392/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-392/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1561
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-588
Configuration saved in ./results/checkpoint-588/config.json
Model weig

In [94]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
fine_tuned_model = DistilBertForSequenceClassification.from_pretrained('./fine-tuned')
def sentiment_analysis(text_list):
    results = []

    for text in tqdm_notebook(text_list):
        # Tokenize the input text
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

        # Get the model's output
        outputs = model(**inputs)

        # Get the predicted class (positive or negative)
        _, predicted_class = torch.max(outputs.logits, dim=1)

        # Convert the predicted class index to sentiment labels
        if predicted_class.item()==0:
            sentiment_label='loss'
        elif predicted_class.item() ==1:
            sentiment_label='draw'
        else:
            sentiment_label = 'win'
        results.append(sentiment_label)

    return results



loading file vocab.txt from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /Users/moksh/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_l

In [95]:
results = sentiment_analysis(text_list)

  0%|          | 0/1561 [00:00<?, ?it/s]

In [111]:
result = sentiment_analysis(['home 2 away 5 a scorer toney'])

  0%|          | 0/1 [00:00<?, ?it/s]

In [112]:
result

['loss']

In [96]:
df_test = pd.DataFrame()

In [97]:
df_test['sentiment']= results

In [98]:
df_test['text'] = text_list